<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg2/B_tf-gradient-tape-linear-clfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

B. Low level tensorflow code - Use gradient tape and low level tensorflow code (do not use keras) to build a linear classifier, svm classifier (hint: adding margin to the of linear  loss) and deep neural network classifier. Provide the necessary metrics for performance of classifier as well as visualizations. Also build a deep neural network by subclassing the keras layer class (see slides - not functional api  and not using denselayer class - you need to define your own MyDenseLayer using subclassing    ) and demonstrate the performance of it.   - check code and links in slides for hints

In [4]:
#https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-3/v-3/1

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
def training_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = models(inputs)
    loss = square_loss(predictions, targets)
  grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b])
  W.assing_sub(grad_loss_wrt_W * learning_rate)
  b.assign_sub(grad_loss_wrt_b * learning_rate)
  return loss

tf.variable is the class meant to manage modifiable state in tensorflow

## Creating tf variables

In [5]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.31270367],
       [ 1.0300264 ],
       [ 0.20949052]], dtype=float32)>


In [6]:
v.assign(tf.ones((3,1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>


In [7]:
v[0, 0].assign(3.)
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>


In [8]:
v.assign_add(tf.ones((3,1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [2.],
       [2.]], dtype=float32)>

In [17]:
a = tf.ones((2,2))
b = tf.square(a)
c = tf.sqrt(a)
d = b + c
e = tf.matmul(a,b)
e *= d

In [16]:
a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [14]:
b

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([1., 1.])>

## Using GradientTape on simple probs

In [20]:
# A tf variable maintains shared, persistent state manipulated by a program.
input_var = tf.Variable(initial_value=3.)

with tf.GradientTape() as tape:
  result = tf.square(input_var)
gradient = tape.gradient(result, input_var)

# https://www.tensorflow.org/api_docs/python/tf/print -- addtl formats and
# piping to stderr
tf.print(input_var)
tf.print(result)
tf.print(gradient)

3
9
6


In [21]:
input_const = tf.constant(3.)
with tf.GradientTape() as tape:
  tape.watch(input_const)
  result = tf.square(input_const)
gradient = tape.gradient(result, input_const)

tf.print(input_const)
tf.print(result)
tf.print(gradient)

3
9
6


In [24]:
# compute second-order gradients
time = tf.Variable(1.)
with tf.GradientTape() as outer_tape:
  with tf.GradientTape() as inner_tape:
    position = 4.9 * time ** 2
  speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)

tf.print(position)
tf.print(speed)
tf.print(acceleration)


4.9
9.8
9.8


## Linear Classifier with GradientTape

In [26]:
num_samples_per_class = 1000
negative_samples = np.random.multivariate_normal(mean=[0, 3], cov=[[1, 0.5], [0.5, 1]], size=num_samples_per_class)
positive_samples = np.random.multivariate_normal(mean=[3, 0], cov=[[1, 0.5], [0.5, 1]], size=num_samples_per_class)

In [ ]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
